In [1]:
#找出各類特徵，以作為機器學習的向量
import codecs
import os
from collections import defaultdict

out_path = "D:\\課業相關\\論文資料\\SVM\\language_feature\\三版\\"

n_rule = ['(Na)','(Nb)','(Nc)','(Ncd)','(Nd)','(Nv)']
v_rule = ['(VA)','(VAC)','(VB)','(VC)','(VCL)','(VD)','(VE)','(VF)','(VG)','(VH)','(VHC)','(VI)','(VJ)','(VK)','(VL)']
d_rule = ['(Da)','(Dfa)','(Dfb)','(Di)','(Dk)','(D)']

In [7]:
#平衡語料庫/自由中國的高頻詞、標點符號、2gram及3gram
#input_path = "D:\\課業相關\\論文資料\\SCS2\\"
input_path = "D:\\課業相關\\論文資料\\雷震處理資料\\source\\自由中國(2nd)\\自由中國-非文藝類\\"

SCS_dic = defaultdict(int)
SCS_category = defaultdict(int)
SCS_bigram = defaultdict(int)
SCS_trigram = defaultdict(int)

for index,file in enumerate(os.listdir(input_path)):
    if index % 3000 == 0:
        print (index)
    with codecs.open(input_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        
        for word in content:
            if '(' in word:
                if 'CATEGORY' in word.split('(')[1]:
                    SCS_category[word.split('(')[0]] += 1
                else:
                    SCS_dic[word.split('(')[0]] += 1
        
        def format_check(word):
            if '(' in word and 'CATEGORY' not in word and word != '' and word[0] != '(':
                return True
            return False
        
        for i in range(len(content)-1):
            if format_check(content[i]) and format_check(content[i+1]):
                SCS_bigram[content[i].split('(')[0]+content[i+1].split('(')[0]] += 1
        
        for i in range(len(content)-2):
            if format_check(content[i]) and format_check(content[i+1]) and format_check(content[i+2]):
                SCS_trigram[content[i].split('(')[0]+content[i+1].split('(')[0]+content[i+2].split('(')[0]] += 1
                
SCS_dic_sort = sorted(SCS_dic.items(), key=lambda d:d[1], reverse = True)    
SCS_category_sort = sorted(SCS_category.items(), key=lambda d:d[1], reverse = True) 

with codecs.open(out_path+'自由中國前100高頻詞.csv','wb','utf8') as g:
    for i,e in enumerate(SCS_dic_sort):
        if i == 100:
            break
        g.write(e[0]+','+str(e[1])+'\r\n')

with codecs.open(out_path+'自由中國前100標點符號.csv','wb','utf8') as g:
    for i,e in enumerate(SCS_category_sort):
        if i == 100:
            break
        g.write(e[0]+','+str(e[1])+'\r\n')

SCS_bigram_sort = sorted(SCS_bigram.items(), key=lambda d:d[1], reverse = True)    
SCS_trigram_sort = sorted(SCS_trigram.items(), key=lambda d:d[1], reverse = True) 

with codecs.open(out_path+'自由中國前100個bigram.csv','wb','utf8') as g:
    for i,e in enumerate(SCS_bigram_sort):
        if i == 100:
            break
        g.write(e[0]+','+str(e[1])+'\r\n')

with codecs.open(out_path+'自由中國前100個trigram.csv','wb','utf8') as g:
    for i,e in enumerate(SCS_trigram_sort):
        if i == 100:
            break
        g.write(e[0]+','+str(e[1])+'\r\n')
        
print ('END')

0
3000
END


In [6]:
#平衡語料庫/自由中國的詞性組合、情態+V組合、否定&程度+N組合
#input_path = "D:\\課業相關\\論文資料\\SCS2\\"
input_path = "D:\\課業相關\\論文資料\\雷震處理資料\\source\\自由中國(2nd)\\自由中國-非文藝類\\"

modal_path = "D:\\課業相關\\論文資料\\謝佳玲情態詞\\word.txt"

modal_feature = []
with codecs.open(modal_path,'rb','utf8') as f:
    content = f.readlines()
    
    for line in content:
        line = line.strip()
        if line != '':
            modal_feature.append(line)

n_n = defaultdict(int)
n_v = defaultdict(int)
d_v = defaultdict(int)
vh_n = defaultdict(int)

modal_v = defaultdict(int)

for index,file in enumerate(os.listdir(input_path)):
    if index % 3000 == 0:
        print (index)
    with codecs.open(input_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        
        def format_check(word):
            if '(' in word and 'CATEGORY' not in word and word != '' and word[0] != '(' and word.split('(')[1] != '':
                return True
            return False
        
        def pos_check(word1,word2,pos1,pos2):
            if word1.split('(')[1][0] == pos1 and word2.split('(')[1][0] == pos2:
                return True
            return False
        
        def n_check(word):
            if word.split('(')[1] == 'Ncd)':
                if len(word.split('(')[0]) >= 2:
                    return True
                return False
            return True
        
        for i in range(len(content)-1):
            if format_check(content[i]) and format_check(content[i+1]): #先無視[+vrr]之類，詞性無例外
                '''if pos_check(content[i],content[i+1],'N','N') \
                    and content[i].split('(')[1].split(')')[0] not in ['Nf','Ng','Nep','Neqa','Neu','Nes'] \
                    and content[i+1].split('(')[1].split(')')[0] != 'Neu':
                    n_n[content[i]+content[i+1]] += 1
                elif pos_check(content[i],content[i+1],'N','V') \
                    and content[i].split('(')[1].split(')')[0] not in ['Nf','Ng','Nep','Neqa','Neu','Nes']:
                    n_v[content[i]+content[i+1]] += 1
                elif pos_check(content[i],content[i+1],'D','V') and content[i].split('(')[1].split(')')[0] != 'DE':
                    d_v[content[i]+content[i+1]] += 1
                elif content[i].split('(')[1][:2] == 'VH' and content[i+1].split('(')[1][0] == 'N' \
                    and content[i+1].split('(')[1].split(')')[0] != 'Neu':
                    vh_n[content[i]+content[i+1]] += 1
                    
                if content[i].split('(')[0] in modal_feature and content[i+1].split('(')[1][0] == 'V' \
                    and content[i].split('(')[1].split(')')[0] != 'DE':
                    modal_v[content[i]+content[i+1]] += 1'''
                
                if n_check(content[i]) and n_check(content[i+1]) and '('+content[i].split('(')[1] in n_rule \
                    and '('+content[i+1].split('(')[1] in n_rule:
                    n_n[content[i]+content[i+1]] += 1
                elif n_check(content[i]) and '('+content[i].split('(')[1] in n_rule \
                    and '('+content[i+1].split('(')[1] in v_rule:
                    n_v[content[i]+content[i+1]] += 1
                elif '('+content[i].split('(')[1] in d_rule and '('+content[i+1].split('(')[1] in v_rule: 
                    d_v[content[i]+content[i+1]] += 1
                elif n_check(content[i+1]) and content[i].split('(')[1][:2] == 'VH' \
                    and '('+content[i+1].split('(')[1] in n_rule: 
                    vh_n[content[i]+content[i+1]] += 1
                
                if content[i].split('(')[0] in modal_feature and '('+content[i+1].split('(')[1] in v_rule \
                    and content[i].split('(')[1].split(')')[0] != 'DE':
                    modal_v[content[i]+content[i+1]] += 1
                    
def dic_sort(dic):
    return sorted(dic.items(), key=lambda d:d[1], reverse = True)

def sort_print(file,dic):
    with codecs.open(out_path+file,'wb','utf8') as g:
        for i,e in enumerate(dic_sort(dic)):
            if i == 100:
                break
            g.write(e[0]+','+str(e[1])+'\r\n')

'''sort_print('平衡語料庫前100個N+N.csv',n_n)
sort_print('平衡語料庫前100個N+V.csv',n_v)
sort_print('平衡語料庫前100個D+V.csv',d_v)
sort_print('平衡語料庫前100個VH+N.csv',vh_n)
sort_print('平衡語料庫前100個情態詞+V.csv',modal_v)'''

sort_print('自由中國前100個N+N.csv',n_n)
sort_print('自由中國前100個N+V.csv',n_v)
sort_print('自由中國前100個D+V.csv',d_v)
sort_print('自由中國前100個VH+N.csv',vh_n)
sort_print('自由中國前100個情態詞+V.csv',modal_v)

print ('END')

0
3000
END


In [10]:
#否定及程度特徵向量擷取
input_path = "D:\\課業相關\\論文資料\\SVM\\language_feature\\未處理\\"

dfa = []

#with codecs.open(input_path+'平衡語料庫程度.csv','rb','utf8') as f:
with codecs.open(input_path+'自由中國程度.csv','rb','utf8') as f:
    head = f.readline()
    content = f.readlines()
    dfa = [(i.strip().split(',')[0],i.strip().split(',')[1]) for i in content \
           if i.strip() != '' and len(i.strip().split(')')) == 3]

#with codecs.open(out_path+'平衡語料庫前100個程度詞組合.csv','wb','utf8') as g:
with codecs.open(out_path+'自由中國前100個程度詞組合.csv','wb','utf8') as g:
    for i,(c,f) in enumerate(dfa):
        if i >= 100:
            break
        g.write(c+','+str(f)+'\r\n')
print ('END')

END


In [25]:
#豐富度比較
import codecs
import os
from collections import OrderedDict,defaultdict,Counter

condicate_author = ['雷震','殷海光','夏道平','龍平甫','蔣勻田','傅正','朱伴耘','胡適','羅鴻詔']

print (' '.join([str(i)+'.'+e for i,e in enumerate(condicate_author)])) #印出作者
print ()

condicate_author_path = "D:\\課業相關\\論文資料\\SVM\\condicate_author\\"

author_richness = defaultdict(list)

X = []
y = []

for file in os.listdir(condicate_author_path):
    with codecs.open(condicate_author_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        content = [i.split('(')[0] for i in content]
        author_richness[file.split('_')[0]].append(len(Counter(content))/len(content))
        X.append(len(Counter(content))/len(content))
        y.append(file.split('_')[0])
        
author_richness = OrderedDict(sorted(author_richness.items(), key=lambda t: len(t[1]),reverse=True))
        
'''for i,e in author_richness.items():
    print (i,len(e))
    print ('max:',max(e))
    print ('min:',min(e))
    print ('mean',sum(e)/len(e))
    print ()'''
    
unknow_author_path = "D:\\課業相關\\論文資料\\SVM\\unknow_author\\"

for file in os.listdir(unknow_author_path):
    with codecs.open(unknow_author_path+file,'rb','utf8') as f:
        head = f.readline().strip()
        content = [i.split('(')[0] for i in f.readline().strip().split()]
        temp_richness = len(Counter(content))/len(content)
        
        X_m = [abs(temp_richness-i) for i in X]
        
        X_m, y_m = zip(*sorted(zip(X_m, y)))
        the = sum([1/i for i in X_m[:100]])
        
        X_t = [1/(i*the) for i in X_m[:100]]
        y_t = y_m[:100]
        
        predict_richness = defaultdict(float)
        for name,score in zip(y_t,X_t):
            predict_richness[name] += score
        
        print (head)
        print ('預測作者：'+max(predict_richness, key=lambda i: predict_richness[i]))
        print ([round(predict_richness[i],3) for i in condicate_author])
        print ()

0.雷震 1.殷海光 2.夏道平 3.龍平甫 4.蔣勻田 5.傅正 6.朱伴耘 7.胡適 8.羅鴻詔

1954-05-16 10卷 - 10期 這是國民黨反省的時候 社論(雷震)(殷海光)
預測作者：殷海光
[0.232, 0.316, 0.107, 0.035, 0.048, 0.099, 0.011, 0.137, 0.016]

1954-06-16 10卷 - 12期 立法院給憲政開一惡例 社論(雷震)(夏道平)
預測作者：夏道平
[0.175, 0.097, 0.33, 0.021, 0.014, 0.325, 0.006, 0.017, 0.015]

1954-07-01 11卷 - 01期 諾言貴實踐 社論(雷震)(夏道平)
預測作者：夏道平
[0.183, 0.176, 0.302, 0.125, 0.027, 0.076, 0.009, 0.094, 0.009]

1954-10-01 11卷 - 07期 又一個關係憲政的問題 社論(雷震)(夏道平)
預測作者：殷海光
[0.092, 0.739, 0.013, 0.002, 0.003, 0.01, 0.001, 0.139, 0.002]

1958-01-01 18卷 - 01期 彈劾案與調查權 社論(雷震)(夏道平)
預測作者：夏道平
[0.253, 0.049, 0.4, 0.03, 0.013, 0.162, 0.013, 0.056, 0.025]

1958-01-01 18卷 - 01期 為「自治」半月刊橫遭查扣而抗議 社論(雷震)(夏道平)
預測作者：夏道平
[0.149, 0.043, 0.409, 0.063, 0.012, 0.158, 0.022, 0.12, 0.022]

1959-01-01 20卷 - 01期 本刊的十年回顧 社論(雷震)(夏道平)
預測作者：傅正
[0.189, 0.171, 0.125, 0.03, 0.122, 0.225, 0.012, 0.121, 0.003]

1959-12-05 21卷 - 11期 開倒車─走私案移送軍法審判 社論(雷震)(夏道平)
預測作者：雷震
[0.495, 0.127, 0.177, 0.062, 0.015, 0.056, 0.01, 0.05, 0.007]

1959-12-05 21卷 - 1

In [5]:
#input_path = "D:\\課業相關\\論文資料\\SCS2\\"
input_path = "D:\\課業相關\\論文資料\\雷震處理資料\\source\\自由中國(2nd)\\自由中國-非文藝類\\"

com_pos = defaultdict(int)
VH_DE_N = 0

for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        
        def format_check(word):
            if '(' in word and 'CATEGORY' not in word and word != '' and word[0] != '(' and word.split('(')[1] != '':
                return True
            return False
        
        def n_check(word):
            if word.split('(')[1] == 'Ncd)':
                if len(word.split('(')[0]) >= 2:
                    return True
                return False
            return True
        
        for i in range(len(content)-1):
            if format_check(content[i]) and format_check(content[i+1]): #先無視[+vrr]之類，詞性無例外
                
                if n_check(content[i]) and n_check(content[i+1]) and '('+content[i].split('(')[1] in n_rule \
                    and '('+content[i+1].split('(')[1] in n_rule:
                    com_pos['N_N'] += 1
                elif n_check(content[i]) and '('+content[i].split('(')[1] in n_rule \
                    and '('+content[i+1].split('(')[1] in v_rule:
                    com_pos['N_V'] += 1
                elif '('+content[i].split('(')[1] in d_rule and '('+content[i+1].split('(')[1] in v_rule: 
                    com_pos['D_V'] += 1
                elif n_check(content[i+1]) and content[i].split('(')[1][:2] == 'VH' \
                    and '('+content[i+1].split('(')[1] in n_rule: 
                    com_pos['VH_N'] += 1
                else:
                    com_pos[content[i].split('(')[1].split(')')[0]+'_'+content[i+1].split('(')[1].split(')')[0]] += 1
                    
        for i in range(len(content)-2):
            if format_check(content[i]) and format_check(content[i+1]) and format_check(content[i+2]):
                if content[i].split('(')[1][:2] == 'VH' and content[i+1].split('(')[1][:2] == 'DE' \
                    and '('+content[i+2].split('(')[1] in n_rule:
                    VH_DE_N += 1
                
                
def dic_sort(dic):
    return sorted(dic.items(), key=lambda d:d[1], reverse = True)

print ('VH_DE_N',VH_DE_N)
print ()

def sort_print(dic):
    for i,e in enumerate(dic_sort(dic)):
        if i == 30:
            break
        print (e[0],e[1])
        
sort_print(com_pos)

VH_DE_N 61078

N_N 420377
D_V 413814
N_V 292269
DE_Na 284937
Na_DE 167869
Na_D 132363
D_D 128252
Neu_Nf 105988
VH_DE 98058
VC_Na 97546
P_Na 82302
VH_N 80526
Nf_Na 69269
D_P 65096
P_Nc 53310
Na_Ng 45902
Nh_D 44648
Na_Caa 43157
DE_VH 40367
VC_DE 39986
Na_P 39957
Nc_DE 39395
D_SHI 39305
Nep_Nf 37651
Cbb_D 36286
VJ_Na 36028
Neu_Na 33420
DE_VC 32742
Caa_Na 32722
Neqa_Na 31945
